In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader

xs = np.random.randint(1, 100, (1, 2, 3))
print(xs)

print(xs.shape)

xs = torch.tensor(xs, dtype=torch.float32)

print(xs.shape)



dataloader = DataLoader(xs, batch_size=1)

for x in (dataloader):
  print(x)





[[[56 82 81]
  [57 98 11]]]
(1, 2, 3)
torch.Size([1, 2, 3])
tensor([[[56., 82., 81.],
         [57., 98., 11.]]])


In [6]:
from data.datasets import MedicalDecathlonDataset
from omegaconf import OmegaConf
import os

cfg = OmegaConf.load("conf/config.yaml")
task_name = cfg.dataset.task_name
dataset_path = f"{cfg.dataset.base_path}{task_name}/"
images_path = f"{dataset_path}{cfg.dataset.images_subdir}"
labels_path = f"{dataset_path}{cfg.dataset.labels_subdir}"
target_shape = cfg.dataset.target_shape




dataset = MedicalDecathlonDataset(cfg, task_name, images_path, labels_path, target_shape)

labels = set()
for x, y in dataset:
  print(x.shape, y.shape)
  break


AssertionError: Invalid label values!

In [ ]:
from data.datasets import MedicalDecathlonDataset
from omegaconf import OmegaConf
import os
import nibabel as nib

cfg = OmegaConf.load("conf/config.yaml")
task_name = cfg.dataset.task_name
dataset_path = f"{cfg.dataset.base_path}{task_name}/"
images_path = f"{dataset_path}{cfg.dataset.images_subdir}"
labels_path = f"{dataset_path}{cfg.dataset.labels_subdir}"
target_shape = cfg.dataset.target_shape


label_path = os.path.join(labels_path, sorted(os.listdir(labels_path))[0])

label = nib.load(label_path).get_fdata() # Shape: [W, H, D]
print(label.min(), label.max())
print(label.shape)




0.0 3.0
(240, 240, 155)
